<a href="https://colab.research.google.com/github/tobatsmr/real-test-task-SQL-1-/blob/main/Loko_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!add-apt-repository -y ppa:sergey-dryabzhinsky/packages
!apt update
!apt install sqlite3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/sergey-drya

### Перезапустить среду выполнения

In [15]:
import sqlite3
print(sqlite3.sqlite_version) # 3.40.1

3.40.1


In [2]:
import pandas as pd
import numpy as np


https://drive.google.com/file/d/1OJmSIPfdcEGwEVJLj28VhUeHEobbTLPq/view?usp=sharing

In [3]:
! gdown 1OJmSIPfdcEGwEVJLj28VhUeHEobbTLPq

Downloading...
From: https://drive.google.com/uc?id=1OJmSIPfdcEGwEVJLj28VhUeHEobbTLPq
To: /content/Loko-bank.zip
100% 53.6k/53.6k [00:00<00:00, 32.1MB/s]


In [4]:
! unzip -o /content/Loko-bank.zip -d /content/loko-bank 

Archive:  /content/Loko-bank.zip
  inflating: /content/loko-bank/accounts.xlsx  
  inflating: /content/loko-bank/blacklist.csv  
  inflating: /content/loko-bank/card_status.xlsx  
  inflating: /content/loko-bank/cards.xlsx  
  inflating: /content/loko-bank/catalog.xlsx  
  inflating: /content/loko-bank/comms.xlsx  
  inflating: /content/loko-bank/tel_catalog.xlsx  


In [5]:
accounts = pd.read_excel('/content/loko-bank/accounts.xlsx')
blacklist = pd.read_csv('/content/loko-bank/blacklist.csv')
card_status = pd.read_excel('/content/loko-bank/card_status.xlsx')
cards = pd.read_excel('/content/loko-bank/cards.xlsx')
catalog = pd.read_excel('/content/loko-bank/catalog.xlsx')
tel_catalog = pd.read_excel('/content/loko-bank/tel_catalog.xlsx')
comms = pd.read_excel('/content/loko-bank/comms.xlsx')

In [6]:
con = sqlite3.connect('db')

In [7]:
cur = con.cursor()

In [8]:
accounts.to_sql('accounts',con,index=False,if_exists='replace')
blacklist.to_sql('blacklist',con,index=False,if_exists='replace')
card_status.to_sql('card_status',con,index=False,if_exists='replace')
catalog.to_sql('catalog',con,index=False,if_exists='replace')
cards.to_sql('cards',con,index=False,if_exists='replace')
tel_catalog.to_sql('tel_catalog',con,index=False,if_exists='replace')
comms.to_sql('comms',con,index=False,if_exists='replace')

In [9]:
def select(sql):
  return pd.read_sql(sql,con)

In [10]:
sql = '''with get_table(client_id, hash_id, rest_rub, tarif_name, tel_num, rank_cart, date_activated, neg)
         AS (
        select t.client_id, t.hash_id, u.rest_rub, r.tarif_name, i.tel_num, 
        RANK() OVER(PARTITION BY t.client_id ORDER BY u.rest_rub desc) as rank_cart,
        t.date_activated, i.tel_num % 2
        from cards t

        left join tel_catalog i ON t.client_id=i.client_id
        left join accounts u ON t.acc_num=u.acc_num
        left join card_status y ON t.hash_id=y.hash_id
        left join catalog r ON t.tarif_id=r.tarif_id 
        
        where t.client_id NOT IN (select * from blacklist x) 
        and r.tarif_name = 'amur tiger'
        and t.date_activated BETWEEN '2016-04-01' AND '2016-06-30'
        and u.rest_rub > 1000
        and DATE(u.report_date) = '2016-07-01'
        and y.status = 'open')

        
        select p.client_id, p.hash_id, p.rest_rub, p.tarif_name, p.tel_num, 
        case when p.tel_num % 2 = 1 
        then strftime('%m', p.date_activated) ||', ' || (p.rest_rub*1.0/1000) || ' тыс.руб, ' || substring(p.tel_num, 2, 4) 
        else 'Уважаемый '|| p.client_id || ', ' ||strftime('%m', p.date_activated) ||', ' || (p.rest_rub*1.0/1000) || ' тыс.руб, ' || substring(p.tel_num, 2, 4) end as sms_text
        from get_table p
        where p.rank_cart = 1

        
        '''
select(sql)

,client_id,hash_id,rest_rub,tarif_name,tel_num,sms_text
0,3,31,2100,amur tiger,89879990058,"Уважаемый 3, 04, 2.1 тыс.руб, 9879"
1,10,10,1100,amur tiger,89879990191,"04, 1.1 тыс.руб, 9879"
2,13,13,1700,amur tiger,89879990248,"Уважаемый 13, 04, 1.7 тыс.руб, 9879"
3,14,14,1900,amur tiger,89879990267,"04, 1.9 тыс.руб, 9879"
4,15,15,2100,amur tiger,89879990286,"Уважаемый 15, 04, 2.1 тыс.руб, 9879"
5,18,18,1100,amur tiger,89879990343,"04, 1.1 тыс.руб, 9879"
6,19,19,1300,amur tiger,89879990362,"Уважаемый 19, 04, 1.3 тыс.руб, 9879"
7,21,21,1700,amur tiger,89879990400,"Уважаемый 21, 04, 1.7 тыс.руб, 9879"
8,22,22,1900,amur tiger,89879990419,"04, 1.9 тыс.руб, 9879"
9,23,23,2100,amur tiger,89879990438,"Уважаемый 23, 04, 2.1 тыс.руб, 9879"


In [11]:
cur.execute('''create table if not exists  table_result AS

        with get_table(client_id, hash_id, rest_rub, tarif_name, tel_num, rank_cart, date_activated)
         AS (
        select t.client_id, t.hash_id, u.rest_rub, r.tarif_name, i.tel_num, 
        RANK() OVER(PARTITION BY t.client_id ORDER BY u.rest_rub desc) as rank_cart,
        t.date_activated
        from cards t

        left join tel_catalog i ON t.client_id=i.client_id
        left join accounts u ON t.acc_num=u.acc_num
        left join card_status y ON t.hash_id=y.hash_id
        left join catalog r ON t.tarif_id=r.tarif_id 
        
        where t.client_id NOT IN (select * from blacklist x) 
        and r.tarif_name = 'amur tiger'
        and t.date_activated BETWEEN '2016-04-01' AND '2016-06-30'
        and u.rest_rub > 1000
        and DATE(u.report_date) = '2016-07-01'
        and y.status = 'open')

        select p.client_id, p.hash_id, p.rest_rub, p.tarif_name, p.tel_num, 
        case when p.tel_num % 2 = 1 
        then strftime('%m', p.date_activated) ||', ' || (p.rest_rub*1.0/1000) || ' тыс.руб, ' || substring(p.tel_num, 2, 4) 
        else 'Уважаемый '|| p.client_id || ', ' ||strftime('%m', p.date_activated) ||', ' || (p.rest_rub*1.0/1000) || ' тыс.руб, ' || substring(p.tel_num, 2, 4) end as sms_text
        from get_table p
        where p.rank_cart = 1
        ''')

In [12]:
sql = ''' select * from table_result t'''
select(sql)

,client_id,hash_id,rest_rub,tarif_name,tel_num,sms_text
0,3,31,2100,amur tiger,89879990058,"Уважаемый 3, 04, 2.1 тыс.руб, 9879"
1,10,10,1100,amur tiger,89879990191,"04, 1.1 тыс.руб, 9879"
2,13,13,1700,amur tiger,89879990248,"Уважаемый 13, 04, 1.7 тыс.руб, 9879"
3,14,14,1900,amur tiger,89879990267,"04, 1.9 тыс.руб, 9879"
4,15,15,2100,amur tiger,89879990286,"Уважаемый 15, 04, 2.1 тыс.руб, 9879"
5,18,18,1100,amur tiger,89879990343,"04, 1.1 тыс.руб, 9879"
6,19,19,1300,amur tiger,89879990362,"Уважаемый 19, 04, 1.3 тыс.руб, 9879"
7,21,21,1700,amur tiger,89879990400,"Уважаемый 21, 04, 1.7 тыс.руб, 9879"
8,22,22,1900,amur tiger,89879990419,"04, 1.9 тыс.руб, 9879"
9,23,23,2100,amur tiger,89879990438,"Уважаемый 23, 04, 2.1 тыс.руб, 9879"


In [13]:
cur.execute(''' insert into comms (client_id, comm_date, days_pass)
          select t.client_id, date('now'), round(JULIANDAY('now') - JULIANDAY('2016-07-01'))
          from table_result t;
      ''')


In [14]:
sql = ''' select *
          from comms t;
      '''
select(sql)

,client_id,comm_date,days_pass
0,3,2023-01-23,2397.0
1,10,2023-01-23,2397.0
2,13,2023-01-23,2397.0
3,14,2023-01-23,2397.0
4,15,2023-01-23,2397.0
5,18,2023-01-23,2397.0
6,19,2023-01-23,2397.0
7,21,2023-01-23,2397.0
8,22,2023-01-23,2397.0
9,23,2023-01-23,2397.0
